In [54]:
import pandas as pd
import numpy as np
import tensorflow
import theano
import pickle

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals.joblib import parallel_backend

### Loading Dataset

In [55]:
df = pd.read_csv('Churn_Modelling.csv')
X = df.iloc[:, 3:13].values
y = df.iloc[:, 13].values

### Examining Data

In [56]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Creating dummy variables for countries and transforming to array

In [58]:
lbl_ncdr_x_1 = LabelEncoder()
X[:, 1] = lbl_ncdr_x_1.fit_transform(X[:, 1])
lbl_ncdr_x_2 = LabelEncoder()
X[:, 2] = lbl_ncdr_x_2.fit_transform(X[:, 2])
hot_ncdr = OneHotEncoder(categorical_features=[1])
X = hot_ncdr.fit_transform(X).toarray()
X = X[:, 1:]

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:385: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [59]:
X.shape

(10000, 11)

In [60]:
y.shape

(10000,)

### Creating train and test data

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=62)

### Standardizing data

In [62]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

# Creating Classifier from Keras library

## Step 1: Create Function with layers

### Layer 1:

- using __add__ to add first hidden layer
- Adding 6 inputs, since our data has 11 columns
    - Using: ceil(# of columns/2) to get 6 layers
- Using rectifier function for hidden layers
- Input Dim is 11, since we have 11 variables and this is the first layer

### Layer 2:

- No need to add input dim, since it is not the first layer

### Layer 3:

- Using Sigmoid function for the output layer
- Since __y__ is binary, we only need one unit

## Step 2: Add ANN compile to the function

- Using 'adam' optimizer
- Using 'binary_crossentropy' __(log loss)__ to calculate loss, since it is a logistic regression
- Using 'accuracy' as metrics for accuracy

In [27]:
def _classifier():
    classifier = Sequential()
    classifier.add(Dense(units=11, kernel_initializer='uniform',
                         activation='relu', input_dim=11))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='softsign'))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    
    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return classifier

### Running first model

In [28]:
classifier = _classifier()
classifier.fit(X_train, y_train, batch_size=20, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 1s 184us/step - loss: 0.4712 - acc: 0.7976
Epoch 2/100
8000/8000 [==============================] - 1s 131us/step - loss: 0.4123 - acc: 0.8311
Epoch 3/100
8000/8000 [==============================] - 1s 132us/step - loss: 0.4057 - acc: 0.8343
Epoch 4/100
8000/8000 [==============================] - 1s 134us/step - loss: 0.4027 - acc: 0.8348
Epoch 5/100
8000/8000 [==============================] - 1s 162us/step - loss: 0.4013 - acc: 0.8356
Epoch 6/100
8000/8000 [==============================] - 1s 137us/step - loss: 0.3994 - acc: 0.8389
Epoch 7/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.3987 - acc: 0.8355
Epoch 8/100
8000/8000 [==============================] - 1s 134us/step - loss: 0.3973 - acc: 0.8380
Epoch 9/100
8000/8000 [==============================] - 1s 140us/step - loss: 0.3973 - acc: 0.8391
Epoch 10/100
8000/8000 [==============================] - 1s 137us/step - loss: 0.3971 - acc: 0.8386

8000/8000 [==============================] - 1s 155us/step - loss: 0.3239 - acc: 0.8645
Epoch 83/100
8000/8000 [==============================] - 1s 146us/step - loss: 0.3230 - acc: 0.8654
Epoch 84/100
8000/8000 [==============================] - 1s 148us/step - loss: 0.3231 - acc: 0.8650
Epoch 85/100
8000/8000 [==============================] - 1s 145us/step - loss: 0.3231 - acc: 0.8637
Epoch 86/100
8000/8000 [==============================] - 1s 144us/step - loss: 0.3228 - acc: 0.8650
Epoch 87/100
8000/8000 [==============================] - 1s 144us/step - loss: 0.3220 - acc: 0.8649
Epoch 88/100
8000/8000 [==============================] - 1s 151us/step - loss: 0.3220 - acc: 0.8634
Epoch 89/100
8000/8000 [==============================] - 1s 156us/step - loss: 0.3215 - acc: 0.8660
Epoch 90/100
8000/8000 [==============================] - 1s 149us/step - loss: 0.3218 - acc: 0.8637
Epoch 91/100
8000/8000 [==============================] - 1s 156us/step - loss: 0.3214 - acc: 0.8646
Epo

In [29]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [30]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1535,   64],
       [ 211,  190]])

In [31]:
print(f'Accuracy = {((1535+190)/20)}%')

Accuracy = 86.25%


## Creating k-fold  cross validation model

In [33]:
classifier = KerasClassifier(
    build_fn=_classifier, batch_size=10, epochs=100)

### Running K-fold cross validation model

In [37]:
accus = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, n_jobs=-4)

Epoch 1/100
7200/7200 [==============================] - 2s 341us/step - loss: 0.4471 - acc: 0.8124
Epoch 2/100
7200/7200 [==============================] - 2s 264us/step - loss: 0.4086 - acc: 0.8300
Epoch 3/100
7200/7200 [==============================] - 2s 262us/step - loss: 0.4040 - acc: 0.8324
Epoch 4/100
7200/7200 [==============================] - 2s 281us/step - loss: 0.3982 - acc: 0.8394
Epoch 5/100
7200/7200 [==============================] - 2s 284us/step - loss: 0.3868 - acc: 0.8436
Epoch 6/100
7200/7200 [==============================] - 2s 277us/step - loss: 0.3765 - acc: 0.8501
Epoch 7/100
7200/7200 [==============================] - 2s 283us/step - loss: 0.3661 - acc: 0.8522
Epoch 8/100
7200/7200 [==============================] - 2s 284us/step - loss: 0.3581 - acc: 0.8558
Epoch 9/100
7200/7200 [==============================] - 2s 292us/step - loss: 0.3555 - acc: 0.8567
Epoch 10/100
7200/7200 [==============================] - 3s 410us/step - loss: 0.3517 - acc: 0.8557

7200/7200 [==============================] - 2s 274us/step - loss: 0.3276 - acc: 0.8679
Epoch 83/100
7200/7200 [==============================] - 2s 264us/step - loss: 0.3280 - acc: 0.8657
Epoch 84/100
7200/7200 [==============================] - 2s 268us/step - loss: 0.3272 - acc: 0.8667
Epoch 85/100
7200/7200 [==============================] - 2s 265us/step - loss: 0.3271 - acc: 0.8658
Epoch 86/100
7200/7200 [==============================] - 2s 264us/step - loss: 0.3265 - acc: 0.8657
Epoch 87/100
7200/7200 [==============================] - 2s 286us/step - loss: 0.3270 - acc: 0.8651
Epoch 88/100
7200/7200 [==============================] - 2s 293us/step - loss: 0.3269 - acc: 0.8672
Epoch 89/100
7200/7200 [==============================] - 2s 281us/step - loss: 0.3270 - acc: 0.8653
Epoch 90/100
7200/7200 [==============================] - 2s 279us/step - loss: 0.3269 - acc: 0.8635
Epoch 91/100
7200/7200 [==============================] - 2s 281us/step - loss: 0.3266 - acc: 0.8651
Epo

7200/7200 [==============================] - 2s 278us/step - loss: 0.3246 - acc: 0.8668
Epoch 63/100
7200/7200 [==============================] - 2s 265us/step - loss: 0.3248 - acc: 0.8669
Epoch 64/100
7200/7200 [==============================] - 2s 266us/step - loss: 0.3241 - acc: 0.8664
Epoch 65/100
7200/7200 [==============================] - 2s 265us/step - loss: 0.3249 - acc: 0.8649
Epoch 66/100
7200/7200 [==============================] - 2s 273us/step - loss: 0.3241 - acc: 0.8669
Epoch 67/100
7200/7200 [==============================] - 3s 369us/step - loss: 0.3237 - acc: 0.8657
Epoch 68/100
7200/7200 [==============================] - 2s 330us/step - loss: 0.3238 - acc: 0.8668
Epoch 69/100
7200/7200 [==============================] - 2s 268us/step - loss: 0.3227 - acc: 0.8672
Epoch 70/100
7200/7200 [==============================] - 2s 321us/step - loss: 0.3239 - acc: 0.8656
Epoch 71/100
7200/7200 [==============================] - 2s 305us/step - loss: 0.3228 - acc: 0.8661
Epo

7200/7200 [==============================] - 2s 277us/step - loss: 0.3279 - acc: 0.8657
Epoch 43/100
7200/7200 [==============================] - 2s 284us/step - loss: 0.3273 - acc: 0.8653
Epoch 44/100
7200/7200 [==============================] - 2s 280us/step - loss: 0.3276 - acc: 0.8657
Epoch 45/100
7200/7200 [==============================] - 2s 269us/step - loss: 0.3271 - acc: 0.8657
Epoch 46/100
7200/7200 [==============================] - 2s 286us/step - loss: 0.3271 - acc: 0.8667
Epoch 47/100
7200/7200 [==============================] - 2s 288us/step - loss: 0.3266 - acc: 0.8629
Epoch 48/100
7200/7200 [==============================] - 2s 289us/step - loss: 0.3267 - acc: 0.8657
Epoch 49/100
7200/7200 [==============================] - 2s 291us/step - loss: 0.3249 - acc: 0.8678
Epoch 50/100
7200/7200 [==============================] - 2s 291us/step - loss: 0.3253 - acc: 0.8646
Epoch 51/100
7200/7200 [==============================] - 2s 279us/step - loss: 0.3250 - acc: 0.8669
Epo

7200/7200 [==============================] - 2s 275us/step - loss: 0.3918 - acc: 0.8393
Epoch 23/100
7200/7200 [==============================] - 2s 347us/step - loss: 0.3914 - acc: 0.8399
Epoch 24/100
7200/7200 [==============================] - 2s 334us/step - loss: 0.3914 - acc: 0.8375
Epoch 25/100
7200/7200 [==============================] - 3s 368us/step - loss: 0.3918 - acc: 0.8389
Epoch 26/100
7200/7200 [==============================] - 3s 402us/step - loss: 0.3915 - acc: 0.8390
Epoch 27/100
7200/7200 [==============================] - 2s 326us/step - loss: 0.3908 - acc: 0.8408
Epoch 28/100
7200/7200 [==============================] - 3s 350us/step - loss: 0.3908 - acc: 0.8399
Epoch 29/100
7200/7200 [==============================] - 2s 335us/step - loss: 0.3903 - acc: 0.8389
Epoch 30/100
7200/7200 [==============================] - 3s 370us/step - loss: 0.3905 - acc: 0.8393
Epoch 31/100
7200/7200 [==============================] - 2s 288us/step - loss: 0.3903 - acc: 0.8396
Epo

...

7200/7200 [==============================] - 2s 285us/step - loss: 0.3860 - acc: 0.8426
Epoch 44/100
7200/7200 [==============================] - 2s 316us/step - loss: 0.3856 - acc: 0.8417
Epoch 45/100
7200/7200 [==============================] - 2s 293us/step - loss: 0.3861 - acc: 0.8406
Epoch 46/100
7200/7200 [==============================] - 2s 314us/step - loss: 0.3865 - acc: 0.8404
Epoch 47/100
7200/7200 [==============================] - 2s 309us/step - loss: 0.3865 - acc: 0.8406
Epoch 48/100
7200/7200 [==============================] - 2s 280us/step - loss: 0.3859 - acc: 0.8419
Epoch 49/100
7200/7200 [==============================] - 2s 292us/step - loss: 0.3857 - acc: 0.8417
Epoch 50/100
7200/7200 [==============================] - 2s 299us/step - loss: 0.3848 - acc: 0.8428
Epoch 51/100
7200/7200 [==============================] - 2s 281us/step - loss: 0.3850 - acc: 0.8425
Epoch 52/100
7200/7200 [==============================] - 2s 307us/step - loss: 0.3848 - acc: 0.8426
Epo

7200/7200 [==============================] - 2s 282us/step - loss: 0.3905 - acc: 0.8412
Epoch 24/100
7200/7200 [==============================] - 2s 280us/step - loss: 0.3903 - acc: 0.8401
Epoch 25/100
7200/7200 [==============================] - 2s 276us/step - loss: 0.3906 - acc: 0.8412
Epoch 26/100
7200/7200 [==============================] - 2s 290us/step - loss: 0.3897 - acc: 0.8412
Epoch 27/100
7200/7200 [==============================] - 2s 298us/step - loss: 0.3902 - acc: 0.8399
Epoch 28/100
7200/7200 [==============================] - 2s 303us/step - loss: 0.3892 - acc: 0.8407
Epoch 29/100
7200/7200 [==============================] - 2s 304us/step - loss: 0.3893 - acc: 0.8410
Epoch 30/100
7200/7200 [==============================] - 2s 296us/step - loss: 0.3898 - acc: 0.8418
Epoch 31/100
7200/7200 [==============================] - 2s 306us/step - loss: 0.3892 - acc: 0.8421
Epoch 32/100
7200/7200 [==============================] - 2s 321us/step - loss: 0.3889 - acc: 0.8400
Epo

7200/7200 [==============================] - 2s 332us/step - loss: 0.4066 - acc: 0.8322
Epoch 4/100
7200/7200 [==============================] - 2s 275us/step - loss: 0.4028 - acc: 0.8318
Epoch 5/100
7200/7200 [==============================] - 2s 317us/step - loss: 0.4026 - acc: 0.8347
Epoch 6/100
7200/7200 [==============================] - 2s 291us/step - loss: 0.4015 - acc: 0.8340
Epoch 7/100
7200/7200 [==============================] - 2s 301us/step - loss: 0.4009 - acc: 0.8332
Epoch 8/100
7200/7200 [==============================] - 2s 298us/step - loss: 0.3999 - acc: 0.8356
Epoch 9/100
7200/7200 [==============================] - 2s 277us/step - loss: 0.3998 - acc: 0.8333
Epoch 10/100
7200/7200 [==============================] - 2s 285us/step - loss: 0.3994 - acc: 0.8337
Epoch 11/100
7200/7200 [==============================] - 2s 278us/step - loss: 0.3980 - acc: 0.8350
Epoch 12/100
7200/7200 [==============================] - 2s 300us/step - loss: 0.3986 - acc: 0.8364
Epoch 13/

7200/7200 [==============================] - 2s 298us/step - loss: 0.3749 - acc: 0.8433
Epoch 85/100
7200/7200 [==============================] - 3s 370us/step - loss: 0.3700 - acc: 0.8464
Epoch 86/100
7200/7200 [==============================] - 2s 300us/step - loss: 0.3645 - acc: 0.8515
Epoch 87/100
7200/7200 [==============================] - 2s 307us/step - loss: 0.3597 - acc: 0.8537
Epoch 88/100
7200/7200 [==============================] - 3s 355us/step - loss: 0.3549 - acc: 0.8569
Epoch 89/100
7200/7200 [==============================] - 2s 319us/step - loss: 0.3508 - acc: 0.8581
Epoch 90/100
7200/7200 [==============================] - 2s 301us/step - loss: 0.3467 - acc: 0.8593
Epoch 91/100
7200/7200 [==============================] - 2s 304us/step - loss: 0.3439 - acc: 0.8604
Epoch 92/100
7200/7200 [==============================] - 2s 311us/step - loss: 0.3416 - acc: 0.8603
Epoch 93/100
7200/7200 [==============================] - 2s 301us/step - loss: 0.3401 - acc: 0.8604
Epo

7200/7200 [==============================] - 2s 283us/step - loss: 0.3299 - acc: 0.8612
Epoch 65/100
7200/7200 [==============================] - 2s 289us/step - loss: 0.3283 - acc: 0.8611
Epoch 66/100
7200/7200 [==============================] - 2s 289us/step - loss: 0.3292 - acc: 0.8603
Epoch 67/100
7200/7200 [==============================] - 2s 282us/step - loss: 0.3285 - acc: 0.8635
Epoch 68/100
7200/7200 [==============================] - 2s 294us/step - loss: 0.3276 - acc: 0.8622
Epoch 69/100
7200/7200 [==============================] - 2s 281us/step - loss: 0.3268 - acc: 0.8631
Epoch 70/100
7200/7200 [==============================] - 2s 285us/step - loss: 0.3279 - acc: 0.8651
Epoch 71/100
7200/7200 [==============================] - 2s 294us/step - loss: 0.3267 - acc: 0.8637
Epoch 72/100
7200/7200 [==============================] - 2s 294us/step - loss: 0.3269 - acc: 0.8632
Epoch 73/100
7200/7200 [==============================] - 2s 290us/step - loss: 0.3265 - acc: 0.8647
Epo

### Accuracies from k-fold cross validation

In [38]:
accus

array([0.86249999, 0.84625   , 0.85749999, 0.85874999, 0.8475    ,
       0.86249999, 0.81999999, 0.82125   , 0.86874999, 0.85874999])

### Taking the mean of all k-fold cross validation models to get an average from all tests

In [39]:
accu_mean = accus.mean()

In [40]:
accu_mean

0.8503749939426779

### Checking variance of all k-fold cross validation models

In [41]:
accu_var = accus.std()

In [42]:
accu_var

0.01618303432557739

## Adding __Dropout__ to k-fold  cross validation model

In [86]:
def dropout_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=11, kernel_initializer='uniform',
                         activation='relu', input_dim=11))
    classifier.add(Dropout(rate=0.12813308004))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='softsign'))
    classifier.add(Dropout(rate=0.15108253694))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    
    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [87]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [88]:
parameters = {'batch_size':[30,45,60], 'nb_epoch': [90,180,360], 
              'optimizer': ['rmsprop', 'sgd', 'Nadam', 'Adadelta']}

In [89]:
grid_search = GridSearchCV(classifier, parameters, scoring = 'accuracy', cv=10, n_jobs=-4)

In [81]:
grid_search = grid_search.fit(X=X_train, y=y_train)

Epoch 1/1
7200/7200 [==============================] - 2s 327us/step - loss: 0.4930 - acc: 0.7940
Epoch 1/1
7200/7200 [==============================] - 2s 299us/step - loss: 0.4844 - acc: 0.7972
Epoch 1/1
7200/7200 [==============================] - 2s 310us/step - loss: 0.4944 - acc: 0.7935
Epoch 1/1
7200/7200 [==============================] - 2s 327us/step - loss: 0.4876 - acc: 0.7951
Epoch 1/1
7200/7200 [==============================] - 2s 327us/step - loss: 0.4893 - acc: 0.7939
Epoch 1/1
7200/7200 [==============================] - 2s 336us/step - loss: 0.4865 - acc: 0.7947
Epoch 1/1
7200/7200 [==============================] - 2s 327us/step - loss: 0.4890 - acc: 0.7961
Epoch 1/1
7200/7200 [==============================] - 3s 350us/step - loss: 0.4929 - acc: 0.7958
Epoch 1/1
7200/7200 [==============================] - 2s 339us/step - loss: 0.4895 - acc: 0.7931
Epoch 1/1
7200/7200 [==============================] - 2s 340us/step - loss: 0.4924 - acc: 0.7935
Epoch 1/1
7200/7200 

7200/7200 [==============================] - 5s 723us/step - loss: 0.5320 - acc: 0.7958
Epoch 1/1
7200/7200 [==============================] - 5s 726us/step - loss: 0.5273 - acc: 0.7935
Epoch 1/1
7200/7200 [==============================] - 5s 746us/step - loss: 0.5334 - acc: 0.7942
Epoch 1/1
7200/7200 [==============================] - 5s 748us/step - loss: 0.5287 - acc: 0.7963
Epoch 1/1
7200/7200 [==============================] - 5s 735us/step - loss: 0.5264 - acc: 0.7964
Epoch 1/1
7200/7200 [==============================] - 5s 743us/step - loss: 0.5328 - acc: 0.7936
Epoch 1/1
7200/7200 [==============================] - 5s 758us/step - loss: 0.5300 - acc: 0.7946
Epoch 1/1
7200/7200 [==============================] - 5s 753us/step - loss: 0.5908 - acc: 0.7940
Epoch 1/1
7200/7200 [==============================] - 5s 756us/step - loss: 0.5882 - acc: 0.7964
Epoch 1/1
7200/7200 [==============================] - 5s 757us/step - loss: 0.5906 - acc: 0.7946
Epoch 1/1
7200/7200 [=========

7200/7200 [==============================] - 8s 1ms/step - loss: 0.4753 - acc: 0.7964
Epoch 1/1
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4781 - acc: 0.7976
Epoch 1/1
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4756 - acc: 0.7989
Epoch 1/1
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4609 - acc: 0.8088
Epoch 1/1
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4675 - acc: 0.8039
Epoch 1/1
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4690 - acc: 0.7996
Epoch 1/1
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4648 - acc: 0.8132
Epoch 1/1
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4662 - acc: 0.8081
Epoch 1/1
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4706 - acc: 0.7994
Epoch 1/1
7200/7200 [==============================] - 9s 1ms/step - loss: 0.4649 - acc: 0.8094
Epoch 1/1
7200/7200 [=============================

7200/7200 [==============================] - 12s 2ms/step - loss: 0.5652 - acc: 0.7928
Epoch 1/1
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5695 - acc: 0.7960
Epoch 1/1
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5664 - acc: 0.7940
Epoch 1/1
7200/7200 [==============================] - 13s 2ms/step - loss: 0.5649 - acc: 0.7940
Epoch 1/1
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5661 - acc: 0.7953
Epoch 1/1
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5664 - acc: 0.7960
Epoch 1/1
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5700 - acc: 0.7921
Epoch 1/1
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5674 - acc: 0.7940
Epoch 1/1
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5702 - acc: 0.7944
Epoch 1/1
7200/7200 [==============================] - 12s 2ms/step - loss: 0.5640 - acc: 0.7963
Epoch 1/1
7200/7200 [===================

7200/7200 [==============================] - 15s 2ms/step - loss: 0.6143 - acc: 0.7956
Epoch 1/1
7200/7200 [==============================] - 15s 2ms/step - loss: 0.6137 - acc: 0.7960
Epoch 1/1
7200/7200 [==============================] - 15s 2ms/step - loss: 0.6154 - acc: 0.7935
Epoch 1/1
7200/7200 [==============================] - 15s 2ms/step - loss: 0.6144 - acc: 0.7943
Epoch 1/1
7200/7200 [==============================] - 15s 2ms/step - loss: 0.5006 - acc: 0.7938
Epoch 1/1
7200/7200 [==============================] - 15s 2ms/step - loss: 0.4971 - acc: 0.7968
Epoch 1/1
7200/7200 [==============================] - 15s 2ms/step - loss: 0.5040 - acc: 0.7933
Epoch 1/1
7200/7200 [==============================] - 15s 2ms/step - loss: 0.4989 - acc: 0.7967
Epoch 1/1
7200/7200 [==============================] - 15s 2ms/step - loss: 0.5038 - acc: 0.7942
Epoch 1/1
7200/7200 [==============================] - 15s 2ms/step - loss: 0.5010 - acc: 0.7928
Epoch 1/1
7200/7200 [===================

7200/7200 [==============================] - 17s 2ms/step - loss: 0.5170 - acc: 0.7924
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.5175 - acc: 0.7964
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.5169 - acc: 0.7932
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.5154 - acc: 0.7953
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.5156 - acc: 0.7938
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.5157 - acc: 0.7936
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.5189 - acc: 0.7947
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.5200 - acc: 0.7967
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.5165 - acc: 0.7932
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.5186 - acc: 0.7946
Epoch 1/1
7200/7200 [===================

7200/7200 [==============================] - 22s 3ms/step - loss: 0.5964 - acc: 0.7922
Epoch 1/1
7200/7200 [==============================] - 21s 3ms/step - loss: 0.5898 - acc: 0.7946
Epoch 1/1
7200/7200 [==============================] - 22s 3ms/step - loss: 0.5937 - acc: 0.7954
Epoch 1/1
7200/7200 [==============================] - 22s 3ms/step - loss: 0.5959 - acc: 0.7951
Epoch 1/1
7200/7200 [==============================] - 22s 3ms/step - loss: 0.5947 - acc: 0.7933
Epoch 1/1
7200/7200 [==============================] - 22s 3ms/step - loss: 0.5939 - acc: 0.7936
Epoch 1/1
7200/7200 [==============================] - 22s 3ms/step - loss: 0.6302 - acc: 0.7925
Epoch 1/1
7200/7200 [==============================] - 22s 3ms/step - loss: 0.6289 - acc: 0.7960
Epoch 1/1
7200/7200 [==============================] - 22s 3ms/step - loss: 0.6306 - acc: 0.7924
Epoch 1/1
7200/7200 [==============================] - 22s 3ms/step - loss: 0.6297 - acc: 0.7942
Epoch 1/1
7200/7200 [===================

In [82]:
best_params = grid_search.best_params_

In [83]:
best_accu = grid_search.best_score_

In [84]:
best_params

{'batch_size': 17, 'nb_epoch': 64, 'optimizer': 'Adadelta'}

In [85]:
best_accu

0.82975